<a href="https://colab.research.google.com/github/bob8dod/ML-studying/blob/main/ML_DL/20_DL_Model_(GAN_generator%2Cdiscriminator).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model #Model_make new Model
from tensorflow.keras.layers import Input, Dense,Flatten,Dropout,Activation, LeakyReLU
from tensorflow.keras.layers import Reshape, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization

import numpy as np
import tensorflow as tf

In [ ]:
seed=0
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
#Generator Model _ Make fake image
generator = Sequential()
generator.add(Dense(128*7*7,input_dim=100,activation=LeakyReLU(0.2))) #7x7x128
generator.add(BatchNormalization()) # for normalization value
generator.add(Reshape((7,7,128))) #7*7*128 -> 7x7,128 / reshape the data
generator.add(UpSampling2D()) # make 2mulitply size -> 14x14, 128
generator.add(Conv2D(64,kernel_size=5, padding='same')) #14x14, 64
generator.add(BatchNormalization()) # for normalization value
generator.add(Activation(LeakyReLU(0.2)))
generator.add(UpSampling2D()) # make 2mulitply size -> 28X28, 64
generator.add(Conv2D(1,kernel_size=5, padding='same',activation='tanh')) #result of fake image -> 28x28, 1

In [ ]:
#Discriminator Model _ make a distinction of fake image
discriminator = Sequential()
discriminator.add(Conv2D(64,kernel_size=5, strides=2,input_shape=(28,28,1),padding='same')) # 13x13, 64
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128,kernel_size=5,strides=2,padding='same')) #5x5, 128
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten()) #5x5, 128 -> 5*5*128
discriminator.add(Dense(1,activation='sigmoid')) # result of distinction
discriminator.compile(loss='binary_crossentropy',optimizer='adam')
discriminator.trainable = False #For not saving the change in weight

In [ ]:
#GAN _ generator + discriminator
ginput = Input(shape=(100,)) #Make input _ vector, size of 100
dis_output = discriminator(generator(ginput)) #set output
gan = Model(ginput, dis_output) # Make Model _ connect generator and discriminator
gan.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
gan.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
sequential_5 (Sequential)    (None, 28, 28, 1)         865281    
_________________________________________________________________
sequential_6 (Sequential)    (None, 1)                 212865    
Total params: 1,078,146
Trainable params: 852,609
Non-trainable params: 225,537
_________________________________________________________________


In [ ]:
#Train GAN
def gan_train(epochs,batch_size,savingpoint):
  (X_train, _), (_, _) = mnist.load_data()  # We use only the X_train Images
  X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32') # Reshape!
  X_train = (X_train - 127.5) / 127.5  #set image's pixel, -1 ~ +1

  true = np.ones((batch_size, 1)) #True label
  fake = np.zeros((batch_size, 1)) #Fake label

  for i in range(epochs + 1): #Train
    #train discriminator through real images
    idx = np.random.randint(0, X_train.shape[0],batch_size) # set random index
    imgs = X_train[idx] #real images
    d_loss_real = discriminator.train_on_batch(imgs,true) #Train! to 1

    #train discriminator through fake images
    noise = np.random.normal(0,1,(batch_size,100)) #set noise with random value, row:batch_size, column: 100_input size
    gen_imgs = generator.predict(noise) #fake images
    d_loss_fake = discriminator.train_on_batch(gen_imgs,fake) #Train! to 0

    d_loss = np.add(d_loss_real,d_loss_fake)/2 #discriminator loss
    g_loss = gan.train_on_batch(noise,true) #GAN loss
    if i%200==0:
            print('epoch:%d' % i, ' d_loss:%.4f' % d_loss, ' g_loss:', g_loss)

gan_train(2000, 32, 200)

epoch:0  d_loss:0.7239  g_loss: [1.148545265197754, 0.34375]
epoch:200  d_loss:0.3169  g_loss: [2.0236964225769043, 0.125]
epoch:400  d_loss:0.2287  g_loss: [3.0745723247528076, 0.03125]
epoch:600  d_loss:0.5653  g_loss: [1.276777744293213, 0.375]
epoch:800  d_loss:0.3586  g_loss: [2.1179730892181396, 0.0625]
epoch:1000  d_loss:0.4773  g_loss: [1.999975323677063, 0.25]
epoch:1200  d_loss:0.3201  g_loss: [1.8208882808685303, 0.1875]
epoch:1400  d_loss:0.4739  g_loss: [1.8242599964141846, 0.09375]
epoch:1600  d_loss:0.4960  g_loss: [1.7764948606491089, 0.15625]
epoch:1800  d_loss:0.6077  g_loss: [1.6049537658691406, 0.15625]
epoch:2000  d_loss:0.4040  g_loss: [1.6755828857421875, 0.15625]
